In [1]:
import os
os.environ['XLA_FLAGS'] = '--xla_force_host_platform_device_count=12'

%load_ext autoreload
%autoreload 2

from brax import envs
from brax.io import html
from brax.training import normalization


import flax
import jax
import jax.numpy as jnp
from brax.envs import create_fn

from IPython.display import HTML, clear_output

import optax

import matplotlib.pyplot as plt
import numpy as np

from controllers import GruController
from common import do_local_apg, add_noise

from functools import partial

jax.config.update('jax_platform_name', 'cpu')

def visualize(sys, qps, height=480):
  """Renders a 3D visualization of the environment."""
  return HTML(html.render(sys, qps, height=height))

len(jax.devices())

12

In [2]:
episode_length = 1000
action_repeat = 1
batch_size = jax.local_device_count()
noise_std = 0.1
apg_epochs = 10
normalize_observations=True

env_name = "fast"  # @param ['ant', 'humanoid', 'fetch', 'grasp', 'halfcheetah', 'walker2d, 'ur5e', 'reacher', bball_1dof]
env_fn = create_fn(env_name = env_name, action_repeat=action_repeat, auto_reset=False)
env = env_fn()

In [3]:
key = jax.random.PRNGKey(0)
reset_keys = jax.random.split(key, num=jax.local_device_count())
_, model_key = jax.random.split(reset_keys[0])
noise_keys = jax.random.split(model_key, num=jax.local_device_count())

In [4]:
policy = GruController(env.observation_size,env.action_size)
normalizer_params, obs_normalizer_update_fn, obs_normalizer_apply_fn = normalization.create_observation_normalizer(
          env.observation_size, normalize_observations, num_leading_batch_dims=1)

add_noise_pmap = jax.pmap(add_noise, in_axes=(None,None,0))
do_apg_pmap = jax.pmap(do_local_apg, in_axes = (None,None,None,None,0,0), static_broadcasted_argnums=(0,1,2))


init_states = jax.pmap(env.reset)(reset_keys)
x0 = init_states.obs
h0 = jnp.zeros(env.observation_size)

policy_params = policy.init(model_key, h0, x0)

In [5]:
#rewards, obs, acts, states = do_rnn_rollout(policy_params, normalizer_params, env.reset(key), policy, obs_normalizer_apply_fn, env.step)

In [9]:
%%time
for i in range(1):
    noise_keys = jax.random.split(noise_keys[0], num=jax.local_device_count())
    train_keys = jax.random.split(noise_keys[0], num=jax.local_device_count())

    policy_params_with_noise, noise = add_noise_pmap(policy_params, noise_std, noise_keys)
    
    policy_params_trained, normalizer_params_all, rewards_lists = do_apg_pmap(apg_epochs, env_fn, policy.apply, normalizer_params, policy_params_with_noise, train_keys)
    
    top_idx = sorted(range(len(rewards_lists)), key=lambda k: rewards_lists[k][-1], reverse=True)

    _, params_def = jax.tree_flatten(policy_params)
    params_flat, _ = jax.tree_flatten(policy_params_trained)

    top_params_flat = [param[top_idx[0]] for param in params_flat]
    top_params = jax.tree_unflatten(params_def, top_params_flat)
    
    policy_params = top_params

    print(rewards_lists[top_idx[0]])
    print('--------------------------------------')

ValueError: pmap was requested to map its argument along axis 0, which implies that its rank should be at least 1, but is only 0 (its shape is ())